In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
import sys
print(sys.executable)


c:\Users\nahue\Documents\Ceia\Analisis de Datos\venv\Scripts\python.exe


In [6]:
producto = pd.read_csv("productos.csv", sep="|", encoding="latin1", dtype=str)
producto.to_pickle('producto.pkl')
comercio = pd.read_csv("comercio.csv", sep="|", encoding="latin1", dtype=str)
comercio.to_pickle('comercio.pkl')
sucursal = pd.read_csv("sucursales.csv", sep="|", encoding="latin1", dtype=str)
sucursal.to_pickle('sucursal.pkl')
dato = pd.read_csv("datos.csv", encoding="latin1", dtype=str)
dato.to_pickle('dato.pkl')


In [5]:


archivo_csv = "productos.csv"
encodings_a_probar = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252', 'utf-8-sig']

for enc in encodings_a_probar:
    try:
        print(f"Intentando con el encoding: {enc}")
        df = pd.read_csv(archivo_csv, sep="|", encoding=enc, dtype=str)
        print("¡Éxito! El archivo se leyó correctamente.")
        print("Primeras filas del DataFrame:")
        print(df.head())
        # Si funciona, salimos del bucle
        break
    except UnicodeDecodeError:
        print(f"Falló. El encoding '{enc}' no es el correcto.\n")
    except Exception as e:
        print(f"Ocurrió un error inesperado con {enc}: {e}\n")

Intentando con el encoding: utf-8
¡Éxito! El archivo se leyó correctamente.
Primeras filas del DataFrame:
  id_comercio id_bandera id_sucursal    id_producto productos_ean  \
0          10          1          63  7790770601899             1   
1          10          3         711  7790070318398             1   
2          10          1          44  7622201808860             1   
3          10          2          62  7791293050089             1   
4          10          1         147  7791905023210             1   

                               productos_descripcion  \
0         TOALLA NOSOTRAS CLASICAS C CALENDULA X 16U   
1         FIDEOS DEDALITOS LUCCHETTI BOLSA X 500 GRS   
2        GALLETITAS BOCA DE DAMA TERRABUSI X 170 GRS   
3  ACONDICIONADOR LISO EFECTO BOTOX TRESEMME X 25...   
4                     LAVANDINA ORIGINAL ODEX X 2 LT   

  productos_cantidad_presentacion productos_unidad_medida_presentacion  \
0                               1                                  U

In [6]:
df

,id_comercio,id_bandera,id_sucursal,id_producto,productos_ean,productos_descripcion,productos_cantidad_presentacion,productos_unidad_medida_presentacion,productos_marca,productos_precio_lista,productos_precio_referencia,productos_cantidad_referencia,productos_unidad_medida_referencia,productos_precio_unitario_promo1,productos_leyenda_promo1,productos_precio_unitario_promo2,productos_leyenda_promo2
0,10,1,63,7790770601899,1,TOALLA NOSOTRAS CLASICAS C CALENDULA X 16U,1,UNI,NOSOTRAS,2709.00,169.31,16,UNI,NaN,NaN,NaN,NaN
1,10,3,711,7790070318398,1,FIDEOS DEDALITOS LUCCHETTI BOLSA X 500 GRS,1,UNI,LUCCHETTI,1439.00,2878.00,500,GRM,NaN,NaN,NaN,NaN
2,10,1,44,7622201808860,1,GALLETITAS BOCA DE DAMA TERRABUSI X 170 GRS,1,UNI,TERRABUSI,1059.00,6229.41,170,GRM,NaN,NaN,NaN,NaN
3,10,2,62,7791293050089,1,ACONDICIONADOR LISO EFECTO BOTOX TRESEMME X 25...,1,UNI,TRESEMME,4455.00,17820.00,250,CM3,NaN,NaN,NaN,NaN
4,10,1,147,7791905023210,1,LAVANDINA ORIGINAL ODEX X 2 LT,1,UNI,ODEX,1900.00,950.00,2000,CM3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944406,10,3,442,7798158671182,1,SNACK PERROS GOLOCAN BOCADITOS CARNE CHOCO 100GRS,1,UNI,GOLOCAN,2130.00,2130.00,1,UNI,NaN,NaN,NaN,NaN
2944407,10,1,149,7790895647659,1,AGUA DE MESA SIN GAS BENEDICTINO PET X 1.5 LT,1,UNI,BENEDICTINO,1519.00,1012.67,1500,CM3,NaN,NaN,NaN,NaN
2944408,10,3,669,7791720029992,1,CHOCLO CREMOSO AMARILLO CLASSIC LATA X 300 GRS,1,UNI,CARREFOUR CLASSIC,999.00,3330.00,300,GRM,NaN,NaN,NaN,NaN
2944409,10,3,441,7794529041608,1,TOSTADAS GRUESAS CLASICAS TOSTI X 200 GRS,1,UNI,TOSTI,1700.00,8500.00,200,GRM,NaN,NaN,NaN,NaN


In [7]:
producto = pd.read_csv("productos.csv", sep="|", encoding="latin1", dtype=str)
producto.sample(25)

,ï»¿id_comercio,id_bandera,id_sucursal,id_producto,productos_ean,productos_descripcion,productos_cantidad_presentacion,productos_unidad_medida_presentacion,productos_marca,productos_precio_lista,productos_precio_referencia,productos_cantidad_referencia,productos_unidad_medida_referencia,productos_precio_unitario_promo1,productos_leyenda_promo1,productos_precio_unitario_promo2,productos_leyenda_promo2
1233126,10,3,561,7793147001025,1,CERVEZA BLANCA SCHNEIDER BOTELLA RET X 1 LT,1,UNI,SCHNEIDER,3300.00,3300.00,1000,CM3,NaN,NaN,NaN,NaN
1041929,10,1,217,7790040142527,1,CEREALES FORMIS POPCORN ARCOR X 145 GRS,1,UNI,FORMIS,1679.00,11579.31,145,GRM,NaN,NaN,NaN,NaN
4180,10,1,175,3616958808598,1,JUEGO DE PIZZA DE 15 PIEZAS,1,UNI,GENERICO,15990.00,15990.00,1,UNI,NaN,NaN,NaN,NaN
2858740,10,3,787,7791720017173,1,MADALENAS DE COCO RELL DDL CARREFOUR BOLSA X 2...,1,UNI,CARREFOUR,1890.00,8790.70,215,GRM,NaN,NaN,NaN,NaN
2920380,10,1,18,7791554020196,1,OLEO CALCAREO BENARE X 500 CC,1,UNI,BENARE,4159.00,8318.00,500,CM3,NaN,NaN,NaN,NaN
1447993,10,1,50,7790538010260,1,NACHOS MACRITAS QUESO BOLSA X 150 GRS,1,UNI,MACRITAS,3060.00,20400.00,150,GRM,NaN,NaN,NaN,NaN
890704,10,2,176,7509552874129,1,ACONDICIONADOR COCO HAIR FOOD FRUCTIS X 300 CC,1,UNI,FRUCTIS,7700.00,25666.67,300,CM3,NaN,NaN,NaN,NaN
245718,10,3,686,7791720036747,1,QUESO GOUDA EN TROZO X 200 GRS BULNEZ,1,UNI,S/D,3550.00,17750.00,200,GRM,NaN,NaN,NaN,NaN
1543479,10,2,203,7798256540144,1,QUESOJA 100% VEGETAL SIN TACC X 360 GRS.,1,UNI,S/D,4729.00,13136.11,360,GRM,NaN,NaN,NaN,NaN
1110941,10,3,376,7791720037096,1,OBLEAS VAINILLA BULNEZ X 50 GRS,1,UNI,BULNEZ,490.00,9800.00,50,GRM,NaN,NaN,NaN,NaN


In [ ]:
producto

In [8]:
producto = producto.rename(columns={"ï»¿id_comercio": "id_comercio"})
producto.sample(2)

,id_comercio,id_bandera,id_sucursal,id_producto,productos_ean,productos_descripcion,productos_cantidad_presentacion,productos_unidad_medida_presentacion,productos_marca,productos_precio_lista,productos_precio_referencia,productos_cantidad_referencia,productos_unidad_medida_referencia,productos_precio_unitario_promo1,productos_leyenda_promo1,productos_precio_unitario_promo2,productos_leyenda_promo2
1461892,10,1,162,5000299628034,1,WHISKY IMPORTADO 7 ANOS BALLANTINES BOUR FINX7...,1,UNI,BALLANTINES,50019.00,71455.71,700,CM3,NaN,NaN,NaN,NaN
1014307,10,1,31,7792129004535,1,EDULCORANTE LIQUIDO SACARINA CHUKER X 200 CC,1,UNI,CHUKER,2900.00,14500.00,200,GRM,NaN,NaN,NaN,NaN


In [ ]:
producto['id_bandera'].unique() #array(['1', '3', '2', nan], dtype=object) 

array(['1', '3', '2', nan], dtype=object)

In [7]:
producto = pd.read_pickle('producto.pkl')
comercio = pd.read_pickle('comercio.pkl')
sucursal = pd.read_pickle('sucursal.pkl')
dato = pd.read_pickle('dato.pkl')

In [9]:
dataframes = [
    ("producto", producto),
    ("comercio", comercio),
    ("sucursal", sucursal),
    ("dato", dato)
]



In [12]:
df =producto
df.columns

Index(['ï»¿id_comercio', 'id_bandera', 'id_sucursal', 'id_producto',
       'productos_ean', 'productos_descripcion',
       'productos_cantidad_presentacion',
       'productos_unidad_medida_presentacion', 'productos_marca',
       'productos_precio_lista', 'productos_precio_referencia',
       'productos_cantidad_referencia', 'productos_unidad_medida_referencia',
       'productos_precio_unitario_promo1', 'productos_leyenda_promo1',
       'productos_precio_unitario_promo2', 'productos_leyenda_promo2'],
      dtype='object')

In [13]:
df

,ï»¿id_comercio,id_bandera,id_sucursal,id_producto,productos_ean,productos_descripcion,productos_cantidad_presentacion,productos_unidad_medida_presentacion,productos_marca,productos_precio_lista,productos_precio_referencia,productos_cantidad_referencia,productos_unidad_medida_referencia,productos_precio_unitario_promo1,productos_leyenda_promo1,productos_precio_unitario_promo2,productos_leyenda_promo2
0,10,1,63,7790770601899,1,TOALLA NOSOTRAS CLASICAS C CALENDULA X 16U,1,UNI,NOSOTRAS,2709.00,169.31,16,UNI,NaN,NaN,NaN,NaN
1,10,3,711,7790070318398,1,FIDEOS DEDALITOS LUCCHETTI BOLSA X 500 GRS,1,UNI,LUCCHETTI,1439.00,2878.00,500,GRM,NaN,NaN,NaN,NaN
2,10,1,44,7622201808860,1,GALLETITAS BOCA DE DAMA TERRABUSI X 170 GRS,1,UNI,TERRABUSI,1059.00,6229.41,170,GRM,NaN,NaN,NaN,NaN
3,10,2,62,7791293050089,1,ACONDICIONADOR LISO EFECTO BOTOX TRESEMME X 25...,1,UNI,TRESEMME,4455.00,17820.00,250,CM3,NaN,NaN,NaN,NaN
4,10,1,147,7791905023210,1,LAVANDINA ORIGINAL ODEX X 2 LT,1,UNI,ODEX,1900.00,950.00,2000,CM3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2944406,10,3,442,7798158671182,1,SNACK PERROS GOLOCAN BOCADITOS CARNE CHOCO 100GRS,1,UNI,GOLOCAN,2130.00,2130.00,1,UNI,NaN,NaN,NaN,NaN
2944407,10,1,149,7790895647659,1,AGUA DE MESA SIN GAS BENEDICTINO PET X 1.5 LT,1,UNI,BENEDICTINO,1519.00,1012.67,1500,CM3,NaN,NaN,NaN,NaN
2944408,10,3,669,7791720029992,1,CHOCLO CREMOSO AMARILLO CLASSIC LATA X 300 GRS,1,UNI,CARREFOUR CLASSIC,999.00,3330.00,300,GRM,NaN,NaN,NaN,NaN
2944409,10,3,441,7794529041608,1,TOSTADAS GRUESAS CLASICAS TOSTI X 200 GRS,1,UNI,TOSTI,1700.00,8500.00,200,GRM,NaN,NaN,NaN,NaN


In [11]:
for name, df in dataframes:
    print(f"\n{'='*40}")
    print(f"📊 Análisis del DataFrame: {name}")
    print(f"{'='*40}")
    
    print("👉 Columnas:")
    print(df.columns.tolist())
    
    print("\n👉 Info:")
    df.info()
    
    print("\n👉 Primeras filas:")
    print(df.head())
    
    print("\n👉 Descripción estadística:")
    print(df.describe(include='all'))



📊 Análisis del DataFrame: producto
👉 Columnas:
['ï»¿id_comercio', 'id_bandera', 'id_sucursal', 'id_producto', 'productos_ean', 'productos_descripcion', 'productos_cantidad_presentacion', 'productos_unidad_medida_presentacion', 'productos_marca', 'productos_precio_lista', 'productos_precio_referencia', 'productos_cantidad_referencia', 'productos_unidad_medida_referencia', 'productos_precio_unitario_promo1', 'productos_leyenda_promo1', 'productos_precio_unitario_promo2', 'productos_leyenda_promo2']

👉 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944411 entries, 0 to 2944410
Data columns (total 17 columns):
 #   Column                                Dtype 
---  ------                                ----- 
 0   ï»¿id_comercio                        object
 1   id_bandera                            object
 2   id_sucursal                           object
 3   id_producto                           object
 4   productos_ean                         object
 5   productos_descripcio